In [4]:
#data download from: https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection
!cp /content/drive/MyDrive/Helmet_detection/data/archive.zip /content

cp: cannot stat '/content/drive/MyDrive/Helmet_detection/data/archive.zip': No such file or directory


In [ ]:
#unzip 
!unzip archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/hard_hat_workers0.png  
  inflating: images/hard_hat_workers1.png  
  inflating: images/hard_hat_workers10.png  
  inflating: images/hard_hat_workers100.png  
  inflating: images/hard_hat_workers1000.png  
  inflating: images/hard_hat_workers1001.png  
  inflating: images/hard_hat_workers1002.png  
  inflating: images/hard_hat_workers1003.png  
  inflating: images/hard_hat_workers1004.png  
  inflating: images/hard_hat_workers1005.png  
  inflating: images/hard_hat_workers1006.png  
  inflating: images/hard_hat_workers1007.png  
  inflating: images/hard_hat_workers1008.png  
  inflating: images/hard_hat_workers1009.png  
  inflating: images/hard_hat_workers101.png  
  inflating: images/hard_hat_workers1010.png  
  inflating: images/hard_hat_workers1011.png  
  inflating: images/hard_hat_workers1012.png  
  inflating: images/hard_hat_workers1013.png  
  inflating: images/hard_hat_workers1014.png  
  inflating: images

In [ ]:
#import library
import numpy as np 
import pandas as pd 
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
import os

In [ ]:
#Create directories (txt files for annotation)
!mkdir -p Dataset/labels
!mkdir -p Dataset/images

In [ ]:
#classes
classes = ['helmet','head','person']

In [ ]:
# calculate anchor points and height width of bonding boxes
def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

In [ ]:
def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = '/content/Dataset/labels/' +  img_jpg_file_name + '.txt'
    print(save_file_name)
    #file_path = open(save_file_name, "a+")
    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

In [ ]:
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
  

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_txt_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
files = os.listdir('/content/annotations')
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    print(file_xml[0])
    get_xml_data('/content/annotations', file_xml[0])

Streaming output truncated to the last 5000 lines.
hard_hat_workers4924
/content/Dataset/labels/hard_hat_workers4924.txt
file name:  hard_hat_workers4453.xml
hard_hat_workers4453
/content/Dataset/labels/hard_hat_workers4453.txt
file name:  hard_hat_workers3735.xml
hard_hat_workers3735
/content/Dataset/labels/hard_hat_workers3735.txt
file name:  hard_hat_workers3831.xml
hard_hat_workers3831
/content/Dataset/labels/hard_hat_workers3831.txt
file name:  hard_hat_workers2371.xml
hard_hat_workers2371
/content/Dataset/labels/hard_hat_workers2371.txt
file name:  hard_hat_workers627.xml
hard_hat_workers627
/content/Dataset/labels/hard_hat_workers627.txt
file name:  hard_hat_workers1343.xml
hard_hat_workers1343
/content/Dataset/labels/hard_hat_workers1343.txt
file name:  hard_hat_workers4662.xml
hard_hat_workers4662
/content/Dataset/labels/hard_hat_workers4662.txt
file name:  hard_hat_workers0.xml
hard_hat_workers0
/content/Dataset/labels/hard_hat_workers0.txt
file name:  hard_hat_workers2173.xm

In [ ]:
# split train val test
from sklearn.model_selection import train_test_split
image_list = os.listdir('images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

total = 5000
train : 4000
val   : 500
test  : 500


In [ ]:
def copy_data(file_list, img_labels_root, imgs_source, mode):

    root_file = Path( '/content/Dataset/images/'+  mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    root_file = Path('/content/Dataset/labels/' + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    for file in file_list:               
        img_name = file.replace('.png', '')        
        img_src_file = imgs_source + '/' + img_name + '.png'        
        label_src_file = img_labels_root + '/' + img_name + '.txt'

        #print(img_sor_file)
        #print(label_sor_file)
        # im = Image.open(rf"{img_sor_file}")
        # im.show()

        # Copy image
        DICT_DIR = '/content/Dataset/images/'  + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = '/content/Dataset/labels/' + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

In [ ]:
copy_data(train_list, '/content/Dataset/labels', '/content/images', "train")
copy_data(val_list,   '/content/Dataset/labels', '/content/images', "val")
copy_data(test_list,  '/content/Dataset/labels', '/content/images', "test")

In [ ]:
len(os.listdir('/content/Dataset/labels/train'))

4000

In [ ]:
!zip -r Dataset.zip Dataset

Streaming output truncated to the last 5000 lines.
  adding: Dataset/labels/val/hard_hat_workers629.txt (deflated 72%)
  adding: Dataset/labels/val/hard_hat_workers2872.txt (deflated 77%)
  adding: Dataset/labels/val/hard_hat_workers14.txt (deflated 76%)
  adding: Dataset/labels/val/hard_hat_workers2608.txt (deflated 74%)
  adding: Dataset/labels/val/hard_hat_workers1890.txt (deflated 79%)
  adding: Dataset/labels/val/hard_hat_workers950.txt (deflated 69%)
  adding: Dataset/labels/val/hard_hat_workers4099.txt (deflated 78%)
  adding: Dataset/labels/val/hard_hat_workers1920.txt (deflated 81%)
  adding: Dataset/labels/val/hard_hat_workers1475.txt (deflated 81%)
  adding: Dataset/labels/val/hard_hat_workers1496.txt (deflated 80%)
  adding: Dataset/labels/val/hard_hat_workers4796.txt (deflated 74%)
  adding: Dataset/labels/val/hard_hat_workers4818.txt (deflated 69%)
  adding: Dataset/labels/val/hard_hat_workers4117.txt (deflated 73%)
  adding: Dataset/labels/val/hard_hat_workers3567.txt (d

In [ ]:
!cp Dataset.zip drive/MyDrive/Helmet_detection/data/